&emsp; 2 &ensp; CoLLiE 的使用案例

&emsp; &emsp; 2.1 &ensp; 案例：指令微调 MOSS

&emsp; &emsp; 2.2 &ensp; CoLLiE 的 Config 模块

&emsp; &emsp; 2.3 &ensp; CoLLiE 的 Monitor 模块

&emsp; &emsp; 2.4 &ensp; CoLLiE 的 Evaluator 模块

&emsp; &emsp; 2.5 &ensp; CoLLiE 的 Trainer 模块

## 2. &ensp; CoLLiE 的使用案例

### 2.1 &ensp; 案例：指令微调 MOSS

&ensp; &ensp; 在上一章中，我们已经介绍了 CoLLiE 提出的背景、实现的功能 以及 包含的模块。

&ensp; &ensp; 写个LOMO、ZeRO3、微调MOSS

In [1]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 torchrun --nproc_per_node=4 --standalone example-tutorial-2-one-sentence-overfitting.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
[2023-07-04 15:41:50,606] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-07-04 15:41:50,606] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-07-04 15:41:50,606] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-07-04 15:41:50,606] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================

===================================BUG REPORT===================================Welcome to bitsandbytes. For bug reports, please run

python -m

[2023-07-04 15:44:10,859] [INFO] [utils.py:785:see_memory_usage] DeepSpeedZeRoOffload initialize [begin]
[2023-07-04 15:44:10,860] [INFO] [utils.py:786:see_memory_usage] MA 7.93 GB         Max_MA 11.3 GB         CA 13.06 GB         Max_CA 13 GB 
[2023-07-04 15:44:10,860] [INFO] [utils.py:793:see_memory_usage] CPU Virtual Memory:  used = 22.67 GB, percent = 4.5%
Parameter Offload: Total persistent parameters: 1474560 in 138 params
[2023-07-04 15:44:10,970] [INFO] [utils.py:785:see_memory_usage] DeepSpeedZeRoOffload initialize [end]
[2023-07-04 15:44:10,971] [INFO] [utils.py:786:see_memory_usage] MA 7.93 GB         Max_MA 7.93 GB         CA 13.06 GB         Max_CA 13 GB 
[2023-07-04 15:44:10,971] [INFO] [utils.py:793:see_memory_usage] CPU Virtual Memory:  used = 22.67 GB, percent = 4.5%
[2023-07-04 15:44:10,972] [INFO] [config.py:960:print] DeepSpeedEngine configuration:
[2023-07-04 15:44:10,973] [INFO] [config.py:964:print]   activation_checkpointing_config  {
    "partition_activations

Training Batch: 0 / 13   0% ━━━━━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- Gradient overflow, skipping step 0
Gradient overflow, skipping step 0
Gradient overflow, skipping step 0
[LOMO] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 4294967296, 
reducing to 2147483648
Training Epoch: 0 / 1    0% ━━━━━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- 
Gradient overflow, skipping step 0━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- 
Training Epoch: 0 / 1    0% ━━━━━━━━━━━━━━━━━━━━━━━ -- it./s 0:00:00 / -:--:-- 
Training Epoch: 0 / 1    0% ━━━━━━━━━━━━ -- it./s 0:00:15 / -:--:--             
Training Epoch: 0 / 1    0% ━━━━━━━━━━━━ -- it./s 0:00:15 / -:--:--             
Training Batch: 0 / 13   8% ╸━━━━━━━━━━━ -- it./s 0:00:15 / -:--:-- Loss: 5.8906Gradient overflow, skipping step 1
Gradient overflow, skipping step 1
Gradient overflow, skipping step 1
[LOMO] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 2147483648, 
reducing to 1073741824
Training Epoch: 0 / 1    0% ━━━━━

                                       s/it.                                    Gradient overflow, skipping step 6Gradient overflow, skipping step 6

Gradient overflow, skipping step 6
[LOMO] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 67108864, reducing 
to 33554432
Training Epoch: 0 / 1    0% ━━━━━━━━━━ -- it./s   0:01:13 / -:--:--             
Training Batch: 5 / 13  46% ━━━━╸━━━━━ 11.44      0:01:12 / 0:01:21 Loss: 5.8906
Gradient overflow, skipping step 6                               
Training Epoch: 0 / 1    0% ━━━━━━━━━━ -- it./s   0:01:13 / -:--:--             
Training Batch: 5 / 13  46% ━━━━╸━━━━━ 11.44      0:01:12 / 0:01:21 Loss: 5.8906
Training Epoch: 0 / 1    0% ━━━━━━━━━━ -- it./s   0:01:24 / -:--:--             
Training Batch: 6 / 13  46% ━━━━╸━━━━━ 11.44      0:01:24 / 0:01:21 Loss: 5.8906
Training Epoch: 0 / 1    0% ━━━━━━━━━━ -- it./s   0:01:24 / -:--:--             
Training Batch: 6 / 13  54% ━━━━━╺━━━━ 11.44      0:01:24 / 0:01:09 Loss: 5.8906
        

Training Epoch: 0 / 1     0% ━━━━━━━━━ -- it./s   0:02:21 / -:--:--             
Training Batch: 11 / 13  85% ━━━━━━━╸━ 11.45      0:02:21 / 0:00:23 Loss: 5.8906
Training Epoch: 0 / 1     0% ━━━━━━━━━ -- it./s   0:02:21 / -:--:--             
Training Batch: 11 / 13  92% ━━━━━━━━╺ 11.45      0:02:21 / 0:00:12 Loss: 5.8906
                                       s/it.                                    Gradient overflow, skipping step 12
Gradient overflow, skipping step 12
Gradient overflow, skipping step 12
[LOMO] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 1048576, reducing 
to 524288
Training Epoch: 0 / 1     0% ━━━━━━━━━ -- it./s   0:02:21 / -:--:--             
Training Batch: 11 / 13  92% ━━━━━━━━╺ 11.45      0:02:21 / 0:00:12 Loss: 5.8906
Gradient overflow, skipping step 12                               
Training Epoch: 0 / 1     0% ━━━━━━━━━ -- it./s   0:02:21 / -:--:--             
Training Batch: 11 / 13  92% ━━━━━━━━╺ 11.45      0:02:21 / 0:00:12 Loss: 5.8906
Trainin

/remote-home/xrliu/anaconda3/envs/collie/lib/python3.9/site-packages/transformer             
s/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to 
control the generation length. This behaviour is deprecated and will be removed 
from the config in v5 of Transformers -- we recommend using `max_new_tokens` to 
control the maximum length of the generation.
  warnings.warn(
Training Epoch: 0 / 1     0% ━━━━━━━━━ -- it./s   0:03:18 / -:--:--             
Training Batch: 12 / 13 100% ━━━━━━━━━ 11.46      0:02:33 / 0:00:00 Loss: 5.8711
                                       s/it.                                    
[15:48:02] INFO     [' It provides a simple and efficient', ' It    ]8;id=243962;file:///remote-home/xrliu/anaconda3/envs/collie/lib/python3.9/site-packages/collie-1.0.1-py3.9.egg/collie/metrics/decode.py\decode.py]8;;\:]8;id=529903;file:///remote-home/xrliu/anaconda3/envs/collie/lib/python3.9/site-packages/collie-1.0.1-py3.9.egg/collie/metrics/decode.

[15:50:49] INFO     [' It provides a simple and efficient', ' It    ]8;id=97251;file:///remote-home/xrliu/anaconda3/envs/collie/lib/python3.9/site-packages/collie-1.0.1-py3.9.egg/collie/metrics/decode.py\decode.py]8;;\:]8;id=398382;file:///remote-home/xrliu/anaconda3/envs/collie/lib/python3.9/site-packages/collie-1.0.1-py3.9.egg/collie/metrics/decode.py#51\51]8;;\
                    provides a simple and efficient', ' It provides             
                    a simple and efficient', ' It provides a simple             
                    and efficient']                                             
Training Epoch: 0 / 1     0% ━━━━━━━━━ -- it./s   0:06:05 / -:--:--             
Training Batch: 12 / 13 100% ━━━━━━━━━ 11.46      0:02:33 / 0:00:00 Loss: 5.8711
                                       s/it.                                    
Training Epoch: 0 / 1     0% ━━━━━━━━━ -- it./s   0:06:38 / -:--:--             
Training Batch: 12 / 13 100% ━━━━━━━━━ 11.46      0:02:33 

### 2.2 &ensp; CoLLiE 的 Config 模块

config参数列表

| 名称 | 类型 | 默认值 | 描述 |
|:----|:----:|:----:|:----|
| seed | int | 42 | Random seed that will be set at the beginning of training | 
| pp_size | int | 1 | Pipeline parallelism degree. | 
| tp_size | int | 1 | Tensor parallelism degree. | 
| dp_size | int | 1 | Data parallelism degree | 
| pp_partition_method | str | 'parameters' | Partition method for pipeline parallelism. Default is 'parameters'. | 
| train_epochs | int | 100 | Number of training epochs. | 
| eval_per_n_steps | int | 0 | Evaluate every n steps. | 
| eval_per_n_epochs | int | 0 | Evaluate every n epochs. | 
| train_micro_batch_size | int | 1 | Batch size (one step) for training. | 
| gradient_accumulation_steps | int | 1 | Number of gradient accumulation steps. | 
| eval_batch_size | int | 1 | Batch size for evaluation. | 
| checkpointing | bool | True | Whether to use activation checkpointing.
| use_flash | bool | True | Whether to use flash attention.
| dropout | float | 0.0 | Dropout probability. | 
| initization_method | str | "none" | Initialization method. Possible values are 'none', 'normal', 'xavier_normal', 'xavier_uniform', 'kaiming_normal', 'kaiming_uniform', 'orthogonal', 'sparse', 'eye', 'dirac'. Default is 'none'. | 
| initization_method_params | dict | None | Parameters for initialization method.
| low_cpu_mem_usage | bool | True | Tries to not use more than 1x model size in CPU memory (including peak memory) while loading the model. | 
| ds_config | Union[str, dict] | "" | DeepSpeed configuration file. | 
| model_config | PretrainedConfig | PretrainedConfig() | Model configuration. | 
| peft_config | PeftConfig | PeftConfig() | PEFT configuration. | 
| quantization_config | BitsAndBytesConfig | BitsAndBytesConfig() | Configuration parameters for the `bitsandbytes` library | 

In [ ]:
from collie.config import CollieConfig

pretrained_model = "fnlp/moss-moon-003-sft"  

config = CollieConfig.from_pretrained(pretrained_model, trust_remote_code=True)
config.tp_size = 1
config.dp_size = 1
config.pp_size = 1
config.train_epochs = 1
config.eval_per_n_steps = 0
config.eval_per_n_epochs = 1 
config.train_micro_batch_size = 2
config.eval_batch_size = 1

### 2.3 &ensp; CoLLiE 的 Monitor 模块

monitor模块列表

| 名称 | 描述 |
|:----|:----|
| BaseMonitor | 用于记录模型训练过程中的统计信息 |
| StepTimeMonitor | 用来记录每个step的时间 |
| NetworkIOMonitor | 用来记录每个step的网络带宽情况 |
| DiskIOMonitor | 用来记录每个step的硬盘读写情况 |
| TGSMonitor | 用来记录每秒每张 GPU 可训练的 token 数 (token / s / GPU) |
| CPUMemoryMonitor | 用来记录每个step的CPU内存占用 |
| MemoryMonitor | 用来记录每个step的内存占用 |
| LossMonitor | 用来记录每个step的loss |
| EvalMonitor | 用来记录每个step的eval结果，仅支持 int 和 float 类型的结果 |
| LRMonitor | 用来记录每个step的learning rate |

In [ ]:
from collie.utils.monitor import StepTimeMonitor, TGSMonitor, MemoryMonitor, LossMonitor, EvalMonitor

monitors = [
    StepTimeMonitor(config),
    TGSMonitor(config),
    MemoryMonitor(config),
    LossMonitor(config),
    EvalMonitor(config)
]

### 2.4 &ensp; CoLLiE 的 Evaluator 模块

evaluator模块列表

| 名称 | 描述 |
|:----|:----|
| Evaluator | | 
| EvaluatorForPerplexity | | 
| EvaluatorForClassfication | | 
| EvaluatorForGeneration | | 

metric模块列表

| 名称 | 描述 |
|:----|:----|
| BaseMetric | |
| DecodeMetric | |
| AccuracyMetric | |
| PPLMetric | |
| BleuMetric | |
| RougeMetric | |
| ClassifyFPreRecMetric | |

In [ ]:
from collie.controller.evaluator import EvaluatorForPerplexity, EvaluatorForGeneration

from collie.metrics import PPLMetric, DecodeMetric

evaluator_ppl = EvaluatorForPerplexity(
    model=model, config=config, dataset=eval_dataset,
    monitors=[EvalMonitor(config), ], metrics={'ppl': PPLMetric(), }
)

evaluator_decode = EvaluatorForGeneration(
    model=model, config=config, tokenizer=tokenizer, dataset=eval_dataset,
    monitors=[EvalMonitor(config), ], metrics={'decode': DecodeMetric(), }
)

### 2.5 &ensp; CoLLiE 的 Trainer 模块

trainer参数列表

| 名称 | 类型 | 默认值 | 描述 |
|:----|:----:|:----:|:----|
| model | torch.nn.Module | | | 
| config | CollieConfig | | | 
| tokenizer | PreTrainedTokenizerBase | None | | 
| loss_fn | Callable | GPTLMLoss() | | 
| train_fn | Callable | None | | 
| eval_fn | Callable | None | | 
| optimizer | torch.optim.Optimizer | None | | 
| lr_scheduler | LRScheduler, DeepSpeedSchedulerCallable | None | | 
| train_dataset | torch.utils.data.Dataset | None | | 
| eval_dataset | torch.utils.data.Dataset | None | | 
| callbacks | Callback, List[Callback] | None | | 
| train_dataset_collate_fn | Callable | ColliePadder() | | 
| eval_dataset_collate_fn | Callable | ColliePadder(padding_left=True) | | 
| data_provider | BaseProvider | None | | 
| monitors | BaseMonitor | [] | | 
| metrics | Dict | None | | 
| evaluators | List | None | | 

In [ ]:
from collie.controller.trainer import Trainer

trainer = Trainer(
    model=model,config=config,train_dataset=train_dataset,
    loss_fn=GPTLMLoss(-100),optimizer=optimizer,
    monitors=monitors, evaluators=[evaluator_ppl, evaluator_decode]
)

trainer.train()